    URLs:
Scores: http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false
Assists: http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false

In [1]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import numpy as np

In [2]:
def scrape(url):
    page=urllib2.urlopen(url).read()
    soup=BeautifulSoup(page)
    soup.prettify()
    return soup

In [3]:
asst_url=str('http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false')
score_url=str('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false')
urls=[score_url,asst_url]

scrape(score_url)

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml">
<head><script src="http://sports-ak.espn.go.com/sports/optimizely.js"></script><meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>
<meta content="#CC0000" name="theme-color"/>
<script type="text/javascript">    
    if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {
        window.location = 'http://a.m.espn.go.com/mobilecache/general/apps/sc';
    }
</script><title>2015-16 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN</title>
<meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>
<meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>
<meta content="noodp" name="googlebot"/>
<meta content="index, follow" name="robots"/>
<meta content="Disc

In [4]:
def get_cols(soup):
    table_div = soup.find(id='my-players-table')
    table=table_div.find('table')
    table_head=table.find(attrs={'class':'colhead'})
    header_cols=table_head.findAll('td')
    cols = []
    for header_col in header_cols:
        val= header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols

get_cols(scrape(score_url))

[u'PLAYER',
 'POSITION',
 u'TEAM',
 u'GP',
 u'MPG',
 u'PTS',
 u'FGM-FGA',
 u'FG%',
 u'3PM-3PA',
 u'3P%',
 u'FTM-FTA',
 u'FT%']

In [6]:
def get_data(cols, soup):
    players_stats_dicts=[]
    players_stats_array=[]
    table_div = soup.find(id='my-players-table')
    table=table_div.find('table')
    table_rows=table.findAll('tr')
    for row in table_rows:
        if row.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        players_stats_array.append(player_stats)
        player_stats = dict(zip(cols, player_stats))
        players_stats_dicts.append(player_stats)
    return players_stats_dicts
    print players_stats_dicts

get_data(get_cols(scrape(score_url)),scrape(score_url))

[{u'3P%': u'.471',
  u'3PM-3PA': u'5.1-10.9',
  u'FG%': u'.535',
  u'FGM-FGA': u'10.6-19.9',
  u'FT%': u'.906',
  u'FTM-FTA': u'6.0-6.6',
  u'GP': u'8',
  u'MPG': u'33.8',
  u'PLAYER': u'Stephen Curry',
  'POSITION': u'PG',
  u'PTS': u'32.4',
  u'TEAM': u'GS'},
 {u'3P%': u'.240',
  u'3PM-3PA': u'2.6-10.7',
  u'FG%': u'.377',
  u'FGM-FGA': u'8.1-21.6',
  u'FT%': u'.854',
  u'FTM-FTA': u'10.9-12.7',
  u'GP': u'7',
  u'MPG': u'39.4',
  u'PLAYER': u'James Harden',
  'POSITION': u'SG',
  u'PTS': u'29.7',
  u'TEAM': u'HOU'},
 {u'3P%': u'.000',
  u'3PM-3PA': u'0.0-0.4',
  u'FG%': u'.581',
  u'FGM-FGA': u'11.3-19.4',
  u'FT%': u'.808',
  u'FTM-FTA': u'6.0-7.4',
  u'GP': u'7',
  u'MPG': u'35.6',
  u'PLAYER': u'Blake Griffin',
  'POSITION': u'PF',
  u'PTS': u'28.6',
  u'TEAM': u'LAC'},
 {u'3P%': u'.447',
  u'3PM-3PA': u'2.6-5.9',
  u'FG%': u'.503',
  u'FGM-FGA': u'9.6-19.1',
  u'FT%': u'.893',
  u'FTM-FTA': u'6.3-7.0',
  u'GP': u'8',
  u'MPG': u'34.8',
  u'PLAYER': u'Kevin Durant',
  'POSITION':

In [17]:
def get_points_data(url):
    from bs4 import BeautifulSoup
    import urllib2
    import pandas as pd
    import numpy as np
    souped_web=scrape(url)
    columns=get_cols(souped_web)
    points_data=get_data(columns,souped_web)
    points_df=pd.DataFrame.from_dict(points_data)
    points_df.head()
    return points_df

In [27]:
def points_and_assts():
    asst_url=str('http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false')
    score_url=str('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false')
    asst_data=get_points_data(asst_url)
    score_data=get_points_data(score_url)
    joined_data=pd.merge(score_data,asst_data,on='PLAYER')
    joined_data.head()
    return joined_data

In [28]:
points_and_assts()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP_x,MPG_x,PLAYER,POSITION_x,...,AP48M,APG,AST,AST/TO,GP_y,MPG_y,POSITION_y,TEAM_y,TO,TOPG
0,.471,5.1-10.9,.535,10.6-19.9,.906,6.0-6.6,8,33.8,Stephen Curry,PG,...,8.4,5.9,47,2.04,8,33.8,PG,GS,23,2.9
1,.240,2.6-10.7,.377,8.1-21.6,.854,10.9-12.7,7,39.4,James Harden,SG,...,7.0,5.7,40,1.14,7,39.4,SG,HOU,35,5.0
2,.429,3.8-8.8,.467,9.6-20.6,.900,4.5-5.0,8,36.6,Damian Lillard,PG,...,9.5,7.3,58,1.93,8,36.6,PG,POR,30,3.8
3,.359,1.8-4.9,.470,9.6-20.5,.844,4.8-5.6,8,34.8,Russell Westbrook,PG,...,15.0,10.9,87,2.23,8,34.8,PG,OKC,39,4.9
4,.207,0.8-3.6,.500,10.0-20.0,.610,4.5-7.4,8,34.8,LeBron James,SF,...,9.2,6.6,53,2.30,8,34.8,SF,CLE,23,2.9
5,.347,2.1-6.1,.420,7.5-17.9,.864,6.4-7.4,8,35.9,Paul George,SF,...,6.5,4.9,39,1.30,8,35.9,SF,IND,30,3.8
6,.393,1.6-4.0,.443,8.3-18.7,.800,4.6-5.7,7,32.7,Reggie Jackson,PG,...,8.2,5.6,39,1.34,7,32.7,PG,DET,29,4.1
7,.346,1.3-3.7,.451,7.9-17.4,.773,4.9-6.3,7,34.4,Eric Bledsoe,PG,...,8.6,6.1,43,2.15,7,34.4,PG,PHX,20,2.9
8,.257,1.5-5.8,.383,6.8-17.8,.907,6.5-7.2,6,30.2,Isaiah Thomas,PG,...,9.3,5.8,35,1.84,6,30.2,PG,BOS,19,3.2
9,.395,2.1-5.4,.410,6.0-14.6,.809,4.8-5.9,8,34.9,Kyle Lowry,PG,...,9.1,6.6,53,1.96,8,34.9,PG,TOR,27,3.4
